In [3]:
print('Importing packages...')
import json
import sys
from pathlib import Path

from utils import *

from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments

# import keras
# import tensorflow as tf
# from keras.callbacks import CSVLogger, ModelCheckpoint
# from keras.layers import Input
# from keras.models import Model
import torch
from torch import nn

HW_path = Path(r'F:\Daneshgah\Arshad\2_4002\NLP\HW5')

data_folder = HW_path / 'data'
results_folder = HW_path / 'results'

cache_dir = HW_path / 'cache'

# config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", cache_dir=cache_dir)

Importing packages...


In [5]:
## Data
print('Importing Data...')

x_train = np.load(data_folder / 'persica.csv_train_x.npy', allow_pickle=True).tolist()
y_train = np.load(data_folder / 'persica.csv_train_y.npy').tolist()
#x_test = np.load(data_folder / 'persica.csv_test_x.npy', allow_pickle=True).tolist()
#y_test = np.load(data_folder / 'persica.csv_test_y.npy').tolist()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", cache_dir=cache_dir)
max_length = 512 #max([len(tokenizer.tokenize(x)) for x in x_train]) + 2

train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(x_val, truncation=True, padding=True, max_length=max_length)
#test_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=max_length)

print('Creating Datasets...')

import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, y_train)
val_dataset = IMDbDataset(val_encodings, y_val)

Importing Data...


Creating Datasets...


In [7]:
## Model
print('Creating Model...')

hyperparameters = dict(
    epochs=15,
    batch_size=1,
    lr=5e-5,
    model_name="HooshvareLab/bert-fa-base-uncased-clf-digimag",
    warmup_steps=500,
    weight_decay=0.01
)

model_name = hyperparameters['model_name']
batch_size = hyperparameters['batch_size']
lr = hyperparameters['lr']
epochs = hyperparameters['epochs']
warmup_steps = hyperparameters['warmup_steps']
weight_decay = hyperparameters['weight_decay']

model = AutoModel.from_pretrained(model_name, cache_dir=cache_dir, num_labels=11)

Creating Model...


KeyboardInterrupt: 

In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification
model_name= 'HooshvareLab/bert-fa-zwnj-base' #"HooshvareLab/bert-base-parsbert-uncased",
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=11)
model

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [9]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(100000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         